### GET NDVI Image Collection

Dataset Used: https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2_HARMONIZED#bands

#### Imports

In [1]:
import time
from tqdm import tqdm
import ee
import geemap
import geopandas as gpd
import pandas as pd

#### Authentication and Initialize

In [2]:
ee.Authenticate()
ee.Initialize()

#### VARS

In [5]:
DATASET_NAME = "COPERNICUS/S2_HARMONIZED"
START_DATE = "2023-01-01"
END_DATE = "2024-01-01"
BANDS = ["B4", "B3", "B2"]
SCALE = 10
# zones = range(2, 10)
zones = [4, 9]

#### Functions

In [4]:
def mask_s2_clouds(image):
  """Masks clouds in a Sentinel-2 image using the QA band.
  Args:
      image (ee.Image): A Sentinel-2 image.
  Returns:
      ee.Image: A cloud-masked Sentinel-2 image.
  """
  qa = image.select('QA60')

  # Bits 10 and 11 are clouds and cirrus, respectively.
  cloud_bit_mask = 1 << 10
  cirrus_bit_mask = 1 << 11

  # Both flags should be set to zero, indicating clear conditions.
  mask = (
      qa.bitwiseAnd(cloud_bit_mask)
      .eq(0)
      .And(qa.bitwiseAnd(cirrus_bit_mask).eq(0))
  )
  return image.updateMask(mask).select("B.*").copyProperties(image, ["system:time_start"])

  # return image.updateMask(mask).divide(10000)

In [5]:
# Write a function that computes NDVI for an image and add it as band
def addNDVI(image):
    ndvi = image.normalizedDifference(["B8", "B4"]).rename("ndvi") # NIR and Red Bands
    return image.addBands(ndvi)

#### Load Dataset

In [6]:
s2  = ee.ImageCollection(DATASET_NAME)

#### Get NDVI Images

In [7]:
for zone in tqdm(zones):
    shapefile_path = f"/vsizip/../shape_files/zone_{zone}.zip/layers/POLYGON.shp"
    gdf = gpd.read_file(shapefile_path)
    roi = geemap.gdf_to_ee(gdf)
    
    # # Extract the centroid of the ROI and get its coordinates.
    # roi_centroid = roi.geometry().centroid().coordinates().getInfo()
    # roi_coords = [roi_centroid[1], roi_centroid[0]]  # Folium expects [lat, lon]

    filtered = s2.filter(ee.Filter.date(START_DATE, END_DATE)) \
    .filter(ee.Filter.lt("CLOUDY_PIXEL_PERCENTAGE", 30)) \
    .filter(ee.Filter.bounds(roi.geometry())) \
    .map(mask_s2_clouds)

    withNDVI = filtered.map(addNDVI)
    median = withNDVI.median()

    
    # EXPORTING THE MEDIAN IMAGE
    print(f"EXPORTING MEDIAN IMAGES FOR {zone}")
    task_median = ee.batch.Export.image.toDrive(
        image=median.select("ndvi"),  # "ndvi" band for median
        description=f"Median_Image_Zone{zone}",
        fileNamePrefix=f"zone{zone}_median_ndvi_{START_DATE}_{END_DATE}",
        folder="veg_ndvi_roi_median_10m",
        scale=SCALE,
        region=roi.geometry().getInfo()["coordinates"],
        maxPixels=1e10
    )
    task_median.start()
    while task_median.active():
        print("Polling for task (id: {}).".format(task_median.id))
        time.sleep(5)  # Poll every 5 seconds

    
    print(f"Task for Zone {zone}, median image download completed with state {task_median.status()['state']}: ")    
    print("#" * 50)

    # # EXPORTING ALL IMAGES WITH 10m RESOLUTION (refer to dataset)
    # print(f"EXPORTING ALL IMAGES FOR {zone}")
    # image_ids = withNDVI.aggregate_array("system:index").getInfo()
    # print("Total Images: ", len(image_ids))

    # for i, image_id in enumerate(tqdm(image_ids, desc="Exporting Images")):
    #     image = ee.Image(withNDVI.filter(ee.Filter.eq("system:index", image_id)).first())
    #     task = ee.batch.Export.image.toDrive(
    #     **{
    #         "image": image.select("ndvi"),
    #         "description": "Image Export {}".format(i + 1),
    #         "fileNamePrefix": f"zone{zone}_{image.id().getInfo()}",
    #         "folder": "veg_ndvi_roi_10m",
    #         "scale": SCALE,
    #         "region": roi.geometry().getInfo()["coordinates"],  # Use ROI geometry
    #         # "region": image.geometry().bounds().getInfo()["coordinates"],
    #         "maxPixels": 1e10,
    #     })
    #     task.start()
    #     # Monitor the task
    #     while task.active():
    #         print("Polling for image (id: {}).".format(task.id))
    #         time.sleep(5)  # Poll every 5 seconds
        
    #     print(f"Image {i + 1} download completed with state {task.status()['state']}: ")  


    print("#" * 50)

  0%|          | 0/2 [00:00<?, ?it/s]

EXPORTING MEDIAN IMAGES FOR 4
Polling for task (id: GS3ZCYPPXA4MNIIX4O7MDV5X).
Polling for task (id: GS3ZCYPPXA4MNIIX4O7MDV5X).
Polling for task (id: GS3ZCYPPXA4MNIIX4O7MDV5X).
Polling for task (id: GS3ZCYPPXA4MNIIX4O7MDV5X).
Polling for task (id: GS3ZCYPPXA4MNIIX4O7MDV5X).
Polling for task (id: GS3ZCYPPXA4MNIIX4O7MDV5X).
Polling for task (id: GS3ZCYPPXA4MNIIX4O7MDV5X).
Polling for task (id: GS3ZCYPPXA4MNIIX4O7MDV5X).
Polling for task (id: GS3ZCYPPXA4MNIIX4O7MDV5X).


 50%|█████     | 1/2 [00:57<00:57, 57.41s/it]

Task for Zone 4, median image download completed with state COMPLETED: 
##################################################
##################################################
EXPORTING MEDIAN IMAGES FOR 9
Polling for task (id: C5FGFGNAL43CBLJK4MLBSRMN).
Polling for task (id: C5FGFGNAL43CBLJK4MLBSRMN).
Polling for task (id: C5FGFGNAL43CBLJK4MLBSRMN).
Polling for task (id: C5FGFGNAL43CBLJK4MLBSRMN).
Polling for task (id: C5FGFGNAL43CBLJK4MLBSRMN).
Polling for task (id: C5FGFGNAL43CBLJK4MLBSRMN).
Polling for task (id: C5FGFGNAL43CBLJK4MLBSRMN).
Polling for task (id: C5FGFGNAL43CBLJK4MLBSRMN).
Polling for task (id: C5FGFGNAL43CBLJK4MLBSRMN).
Polling for task (id: C5FGFGNAL43CBLJK4MLBSRMN).
Polling for task (id: C5FGFGNAL43CBLJK4MLBSRMN).
Polling for task (id: C5FGFGNAL43CBLJK4MLBSRMN).
Polling for task (id: C5FGFGNAL43CBLJK4MLBSRMN).
Polling for task (id: C5FGFGNAL43CBLJK4MLBSRMN).


100%|██████████| 2/2 [02:14<00:00, 67.36s/it]

Task for Zone 9, median image download completed with state COMPLETED: 
##################################################
##################################################
